In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

In [ ]:
# # changing current directory to the google drive
%cd '/content/drive/My Drive'

# # cloning darknet repo, this will create a folder, darknet
!git clone https://github.com/AlexeyAB/darknet 

# # going to the darknet folder and getting the weights
%cd darknet
!wget http://pjreddie.com/media/files/darknet53.conv.74

# # make changes in the Makefile to use GPU for training
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# # compiling
!make

In [ ]:
%cd '/content/drive/My Drive/darknet'

# unzipping custom data to data folder
! unzip -uq Racing_Bikes/custom_data.zip -d data

# copying the below files to data folder
! cp Racing_Bikes/classes.names data
! cp Racing_Bikes/obj.data.txt data
! cp Racing_Bikes/train.txt data
! cp Racing_Bikes/test.txt data
# copying the config files to cfg folder
! cp Racing_Bikes/yolov3_custom_train.cfg cfg
! cp Racing_Bikes/yolov3_custom_test.cfg cfg

In [ ]:
# begin training by running this cell
%cd '/content/drive/My Drive/darknet'
! ./darknet detector train data/obj.data.txt cfg/yolov3_custom_train.cfg darknet53.conv.74 -dont_show

# this will create weight files in backup folder inside darknet
# when the training is completed, you'll see a file named
#yolov3_custom_train_final.weights

In [ ]:
# # if the training stopped for whatever reason, 
# # just run this cell to restart the training from the last check point
%cd '/content/drive/My Drive/darknet'
! ./darknet detector train data/obj.data.txt cfg/yolov3_custom_train.cfg backup/yolov3_custom_train_last.weights -dont_show

In [ ]:
%cd '/content/drive/My Drive/darknet'

In [ ]:
# to test model with images, upload the images in the darknet folder
# this way you'll only have to write the name of the image instead of the entire path 
! ./darknet detector test data/obj.data.txt cfg/yolov3_custom_test.cfg backup/yolov3_custom_train_final.weights /content/drive/MyDrive/darknet/data/pic5.png -thresh 0.2

# this will create a image, 'predictions.jpg' in the darknet folder

In [ ]:
# to check the predictions
from PIL import Image

im=Image.open('predictions.jpg')
im.resize((640,480))

In [ ]:
#run model on video data
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
net = cv2.dnn.readNet('/content/drive/MyDrive/darknet/backup/yolov3_custom_train_final.weights', '/content/drive/MyDrive/darknet/cfg/yolov3_custom_test.cfg')

classes = []
with open("/content/drive/MyDrive/darknet/data/classes.names", "r") as f:
    classes = f.read().splitlines()

cap = cv2.VideoCapture('/content/drive/MyDrive/darknet/Bike_racing.mkv')
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

while True:
    _, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

    cv2_imshow(img)
    key = cv2.waitKey(1)
    if key==1:
        break

cap.release()
cv2.destroyAllWindows()